# Join Gov DataSets

In [79]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [17]:
df_tse = pd.read_csv('./all_governors_from_tse.csv', sep=';', encoding='iso-8859-1')
df_gazeta = pd.read_csv('./all_governors_from_gazeta.csv', sep=';', encoding='iso-8859-1')
df_all_city = pd.read_csv('./all_cities_bahia_gov.csv', sep=';', encoding='iso-8859-1')
df_gov_by_cities = pd.read_csv('./all_governors_from_gazeta_by_cities.csv', sep=';', encoding='iso-8859-1')

In [18]:
df_tse

,id,nomeUrna,numero,nomeCompleto,descricaoSexo,descricaoCorRaca,grauInstrucao,ocupacao,gastoCampanha1T,gastoCampanha2T,sgUfNascimento,nomeMunicipioNascimento,localCandidatura,fotoUrl,descricaoTotalizacao,nomeColigacao,composicaoColigacao,cargo,totalDeBens,partido,descricaoNaturalidade,sigla,nomePartido,siglaPartido,nrCandidato,dividaCampanha,sobraFinanceira,limiteDeGasto1T,limiteDeGasto2T,valorLimiteDeGastos,totalDespesasContratadas,totalDespesasPagas,doacoesOutrosCandidatosPartigos,fundosPartidarios,fundoEspecial,outrosRecursos,financeiras,estimaveis,totalRecebido,qtdRecebido,percentualRecebido,totalFinanceiro,qtdFinanceiro,percentualFinanceiro,totalEstimados,qtdEstimados,percentualEstimados,totalReceitaPF,qtdReceitaPF,percentualReceitaPF,totalReceitaPJ,qtdReceitaPJ,percentualReceitaPJ,totalPartidos,qtdPartidos,percentualPartidos,totalInternet,qtdInternet,percentualInternet,totalRoni,qtdRoni,percentualRoni,totalProprios,qtdProprios,percentualProprios,totalReceitaOutCand,qtdReceitaOutCand,percentualReceitaOutCand,totalDoacaoFcc,qtdDoacaoFcc,percentualDoacaoFcc,sqPrestadorConta,sqEntregaPrestacao,graphQtdReceitaFin,graphVrReceitaFin,graphQtdReceitaFinFundo,graphVrReceitaFinFundo,graphPercentReceitaFinFundo,graphQtdReceitaFinOutros,graphVrReceitaFinOutros,graphPercentFinOutros,graphQtdReceitaFinFefc,graphVrReceitaFinFefc,graphPercentFinFefc,concentracaoDespesasR,rankingDoadoresR,rankingFornecedoresR
0,50000623171,CÉLIA SACRAMENTO,18,CÉLIA OLIVEIRA DE JESUS SACRAMENTO,FEM.,PRETA,Superior completo,Professor de Ensino Superior,14000000,7000000,SP,SÃO PAULO,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,REDE,**,Governador,1198000.00,Rede Sustentabilidade,SP-SÃO PAULO,REDE,Rede Sustentabilidade,REDE,18,NaN,NaN,14000000.0,7000000.0,14000000.0,50301.15,50301.15,0.0,30000.00,20000.00,301.15,0.0,0.0,50301.15,6,NaN,50301.15,6,100.00,0.00,0,0.00,0.00,0,0.00,0,0,0,50000.00,2,99.40,0,0,0,0,0,0.00,101.15,3,0.2,0.00,0,0.0,200,1,0.40,NaN,NaN,6,50301.15,1,30000.00,59.64,4,301.15,0.60,1,20000.00,39.76,50301.15,50301.15,38950.0
1,50000622082,JOÃO HENRIQUE DURVAL CARNEIRO,28,JOÃO HENRIQUE DE BARRADAS CARNEIRO,MASC.,PARDA,Superior completo,Economista,14000000,7000000,BA,FEIRA DE SANTANA,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,"BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS",PRTB / PSL,Governador,1502885.55,Partido Renovador Trabalhista Brasileiro,BA-FEIRA DE SANTANA,PRTB,Partido Renovador Trabalhista Brasileiro,PRTB,28,NaN,NaN,14000000.0,7000000.0,14000000.0,1000.00,1000.00,0.0,0.00,0.00,1000.00,0.0,0.0,1500.00,2,NaN,1000.00,1,66.67,500.00,1,33.33,1500.00,2,100.00,0,0,0,0.00,0,0.00,0,0,0,0,0,0.00,0.00,0,0.0,0.00,0,0.0,0,0,0.00,NaN,NaN,1,1000.00,0,0.00,0.00,1,1000.00,100.00,0,0.00,0.00,1500.00,1500.00,990.0
2,50000608056,JOÃO SANTANA,15,JOÃO REIS SANTANA FILHO,MASC.,PARDA,Ensino Médio completo,Aposentado (Exceto Servidor Público),14000000,7000000,BA,NAZARÉ,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,PRA MUDAR DE VERDADE,MDB / DC,Governador,320541.42,Movimento Democrático Brasileiro,BA-NAZARÉ,MDB,Movimento Democrático Brasileiro,MDB,15,0.00,7404.65,14000000.0,7000000.0,14000000.0,3379195.35,3379195.35,0.0,908897.35,2470298.00,0.00,0.0,0.0,3386600.00,10,NaN,3386600.00,10,100.00,0.00,0,0.00,0.00,0,0.00,0,0,0,3386600.00,10,100.00,0,0,0,0,0,0.00,0.00,0,0.0,0.00,0,0.0,0,0,0.00,NaN,NaN,10,3386600.00,5,915100.00,27.02,0,0.00,0.00,5,2471500.00,72.98,3379195.35,3386600.00,3195318.0
3,50000600867,MARCOS MENDES,50,MARCOS ANTONIO GUIMARAES MENDES,MASC.,PARDA,Superior completo,Outros,14000000,7000000,BA,SALVADOR,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,PSOL,**,Governador,315000.00,Partido Socialismo e Liberdade,BA-SALVADOR,PSOL,Partido Socialismo e Liberdade,PSOL,50,NaN,NaN,14000000.0,7000000.0,14000000.0,88458.95,88458.95,0.0,0.00,87182.95,1276.00,0.0,0.0,113141.87,10,NaN,88841.87,5,78.52,24300.00,5,21.48,0.00,0,0.00,0,0,0,111865.87,9,98.87,0,0,0,0,0,0.00,0.00,0,0.0,0.00,0,0

In [19]:
df_gazeta

,url_gov,id_tse,name,partido,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento
0,https://especiais.gazetadopovo.com.br/eleicoes...,50000607711,Rui Costa Eleito,PT,75.50%,5096062,"Candidato do PT, Rui Costa obteve 5.096.062 vo...",Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAgente Postal, ...",Rui Costa dos Santos é o atual governador da B...,Rui Costa,Masculino,59 anos,18/01/1963,Economista,Superior completo,Casado(a),Parda,Salvador-BA
1,https://especiais.gazetadopovo.com.br/eleicoes...,50000608319,Zé Ronaldo,DEM,22.26%,1502266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",José Ronaldo de Carvalho é ex-prefeito de Feir...,Zé Ronaldo,Masculino,70 anos,18/07/1951,Administrador,Superior completo,Casado(a),Branca,Paripiranga-BA
2,https://especiais.gazetadopovo.com.br/eleicoes...,50000600867,Marcos Mendes,PSOL,0.66%,44702,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAdministrador, ...",Marcos Antonio Guimarães Mendes é geólogo e po...,Marcos Mendes,Masculino,57 anos,28/02/1965,Outros,Superior completo,Casado(a),Parda,Salvador-BA
3,https://especiais.gazetadopovo.com.br/eleicoes...,50000622082,João Henrique Durval Carneiro,PRTB,0.58%,38960,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\t ...",João Henrique de Barradas Carneiro é candidato...,João Henrique Durval Carneiro,Masculino,62 anos,19/06/1959,Economista,Superior completo,Casado(a),Parda,Feira de Santana-BA
4,https://especiais.gazetadopovo.com.br/eleicoes...,50000608056,João Santana,MDB,0.49%,33266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",João Reis Santana Filho é engenheiro eletricis...,João Santana,Masculino,78 anos,01/02/1944,Aposentado (Exceto Servidor Público),Ensino Médio completo,Casado(a),Parda,Nazaré-BA
5,https://especiais.gazetadopovo.com.br/eleicoes...,50000623171,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP
6,https://especiais.gazetadopovo.com.br/eleicoes...,50000618934,Orlando Andrade,PCO,0.05%,3100,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Divorciado(a),\n\t\t\t\t\t\tProfessor de Ensin...",NaN,Orlando Andrade,Masculino,36 anos,02/05/1985,Agente Postal,Ensino Médio completo,Casado(a),Preta,Feira de Santana-BA


In [22]:
df_gov = pd.merge(df_tse, df_gazeta, how="inner", left_on ="id", right_on="id_tse")

In [23]:
df_gov

,id,nomeUrna,numero,nomeCompleto,descricaoSexo,descricaoCorRaca,grauInstrucao,ocupacao,gastoCampanha1T,gastoCampanha2T,sgUfNascimento,nomeMunicipioNascimento,localCandidatura,fotoUrl,descricaoTotalizacao,nomeColigacao,composicaoColigacao,cargo,totalDeBens,partido_x,descricaoNaturalidade,sigla,nomePartido,siglaPartido,nrCandidato,dividaCampanha,sobraFinanceira,limiteDeGasto1T,limiteDeGasto2T,valorLimiteDeGastos,totalDespesasContratadas,totalDespesasPagas,doacoesOutrosCandidatosPartigos,fundosPartidarios,fundoEspecial,outrosRecursos,financeiras,estimaveis,totalRecebido,qtdRecebido,percentualRecebido,totalFinanceiro,qtdFinanceiro,percentualFinanceiro,totalEstimados,qtdEstimados,percentualEstimados,totalReceitaPF,qtdReceitaPF,percentualReceitaPF,totalReceitaPJ,qtdReceitaPJ,percentualReceitaPJ,totalPartidos,qtdPartidos,percentualPartidos,totalInternet,qtdInternet,percentualInternet,totalRoni,qtdRoni,percentualRoni,totalProprios,qtdProprios,percentualProprios,totalReceitaOutCand,qtdReceitaOutCand,percentualReceitaOutCand,totalDoacaoFcc,qtdDoacaoFcc,percentualDoacaoFcc,sqPrestadorConta,sqEntregaPrestacao,graphQtdReceitaFin,graphVrReceitaFin,graphQtdReceitaFinFundo,graphVrReceitaFinFundo,graphPercentReceitaFinFundo,graphQtdReceitaFinOutros,graphVrReceitaFinOutros,graphPercentFinOutros,graphQtdReceitaFinFefc,graphVrReceitaFinFefc,graphPercentFinFefc,concentracaoDespesasR,rankingDoadoresR,rankingFornecedoresR,url_gov,id_tse,name,partido_y,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento
0,50000623171,CÉLIA SACRAMENTO,18,CÉLIA OLIVEIRA DE JESUS SACRAMENTO,FEM.,PRETA,Superior completo,Professor de Ensino Superior,14000000,7000000,SP,SÃO PAULO,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,REDE,**,Governador,1198000.00,Rede Sustentabilidade,SP-SÃO PAULO,REDE,Rede Sustentabilidade,REDE,18,NaN,NaN,14000000.0,7000000.0,14000000.0,50301.15,50301.15,0.0,30000.00,20000.00,301.15,0.0,0.0,50301.15,6,NaN,50301.15,6,100.00,0.00,0,0.00,0.00,0,0.00,0,0,0,50000.00,2,99.40,0,0,0,0,0,0.00,101.15,3,0.2,0.00,0,0.0,200,1,0.40,NaN,NaN,6,50301.15,1,30000.00,59.64,4,301.15,0.60,1,20000.00,39.76,50301.15,50301.15,38950.0,https://especiais.gazetadopovo.com.br/eleicoes...,50000623171,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP
1,50000622082,JOÃO HENRIQUE DURVAL CARNEIRO,28,JOÃO HENRIQUE DE BARRADAS CARNEIRO,MASC.,PARDA,Superior completo,Economista,14000000,7000000,BA,FEIRA DE SANTANA,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,"BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS",PRTB / PSL,Governador,1502885.55,Partido Renovador Trabalhista Brasileiro,BA-FEIRA DE SANTANA,PRTB,Partido Renovador Trabalhista Brasileiro,PRTB,28,NaN,NaN,14000000.0,7000000.0,14000000.0,1000.00,1000.00,0.0,0.00,0.00,1000.00,0.0,0.0,1500.00,2,NaN,1000.00,1,66.67,500.00,1,33.33,1500.00,2,100.00,0,0,0,0.00,0,0.00,0,0,0,0,0,0.00,0.00,0,0.0,0.00,0,0.0,0,0,0.00,NaN,NaN,1,1000.00,0,0.00,0.00,1,1000.00,100.00,0,0.00,0.00,1500.00,1500.00,990.0,https://especiais.gazetadopovo.com.br/eleicoes...,50000622082,João Henrique Durval Carneiro,PRTB,0.58%,38960,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\t ...",João Henrique de Barradas Carneiro é candidato...,João Henrique Durval Carneiro,Masculino,62 anos,19/06/1959,Economista,Superior completo,Casado(a),Parda,Feira de Santana-BA
2,50000608056,JOÃO SANTANA,15,JOÃO REIS SANTANA FILHO,MASC.,PARDA,Ensino Médio completo,Aposentado (Exceto Servidor Público),1400

In [70]:
df_gov.head(1)

,id,nomeUrna,numero,nomeCompleto,descricaoSexo,descricaoCorRaca,grauInstrucao,ocupacao,gastoCampanha1T,gastoCampanha2T,sgUfNascimento,nomeMunicipioNascimento,localCandidatura,fotoUrl,descricaoTotalizacao,nomeColigacao,composicaoColigacao,cargo,totalDeBens,partido_x,descricaoNaturalidade,sigla,nomePartido,siglaPartido,nrCandidato,dividaCampanha,sobraFinanceira,limiteDeGasto1T,limiteDeGasto2T,valorLimiteDeGastos,totalDespesasContratadas,totalDespesasPagas,doacoesOutrosCandidatosPartigos,fundosPartidarios,fundoEspecial,outrosRecursos,financeiras,estimaveis,totalRecebido,qtdRecebido,percentualRecebido,totalFinanceiro,qtdFinanceiro,percentualFinanceiro,totalEstimados,qtdEstimados,percentualEstimados,totalReceitaPF,qtdReceitaPF,percentualReceitaPF,totalReceitaPJ,qtdReceitaPJ,percentualReceitaPJ,totalPartidos,qtdPartidos,percentualPartidos,totalInternet,qtdInternet,percentualInternet,totalRoni,qtdRoni,percentualRoni,totalProprios,qtdProprios,percentualProprios,totalReceitaOutCand,qtdReceitaOutCand,percentualReceitaOutCand,totalDoacaoFcc,qtdDoacaoFcc,percentualDoacaoFcc,sqPrestadorConta,sqEntregaPrestacao,graphQtdReceitaFin,graphVrReceitaFin,graphQtdReceitaFinFundo,graphVrReceitaFinFundo,graphPercentReceitaFinFundo,graphQtdReceitaFinOutros,graphVrReceitaFinOutros,graphPercentFinOutros,graphQtdReceitaFinFefc,graphVrReceitaFinFefc,graphPercentFinFefc,concentracaoDespesasR,rankingDoadoresR,rankingFornecedoresR,url_gov,id_tse,name,partido_y,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento
0,50000623171,CÉLIA SACRAMENTO,18,CÉLIA OLIVEIRA DE JESUS SACRAMENTO,FEM.,PRETA,Superior completo,Professor de Ensino Superior,14000000,7000000,SP,SÃO PAULO,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,REDE,**,Governador,1198000.0,Rede Sustentabilidade,SP-SÃO PAULO,REDE,Rede Sustentabilidade,REDE,18,NaN,NaN,14000000.0,7000000.0,14000000.0,50301.15,50301.15,0.0,30000.0,20000.0,301.15,0.0,0.0,50301.15,6,NaN,50301.15,6,100.0,0.0,0,0.0,0.0,0,0.0,0,0,0,50000.0,2,99.4,0,0,0,0,0,0.0,101.15,3,0.2,0.0,0,0.0,200,1,0.4,NaN,NaN,6,50301.15,1,30000.0,59.64,4,301.15,0.6,1,20000.0,39.76,50301.15,50301.15,38950.0,https://especiais.gazetadopovo.com.br/eleicoes...,50000623171,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP


In [72]:
df_gov['key_url'] = df_gov['url_gov'].apply(lambda x: x.split('/')[-2])
df_gov.head(10)

,id,nomeUrna,numero,nomeCompleto,descricaoSexo,descricaoCorRaca,grauInstrucao,ocupacao,gastoCampanha1T,gastoCampanha2T,sgUfNascimento,nomeMunicipioNascimento,localCandidatura,fotoUrl,descricaoTotalizacao,nomeColigacao,composicaoColigacao,cargo,totalDeBens,partido_x,descricaoNaturalidade,sigla,nomePartido,siglaPartido,nrCandidato,dividaCampanha,sobraFinanceira,limiteDeGasto1T,limiteDeGasto2T,valorLimiteDeGastos,totalDespesasContratadas,totalDespesasPagas,doacoesOutrosCandidatosPartigos,fundosPartidarios,fundoEspecial,outrosRecursos,financeiras,estimaveis,totalRecebido,qtdRecebido,percentualRecebido,totalFinanceiro,qtdFinanceiro,percentualFinanceiro,totalEstimados,qtdEstimados,percentualEstimados,totalReceitaPF,qtdReceitaPF,percentualReceitaPF,totalReceitaPJ,qtdReceitaPJ,percentualReceitaPJ,totalPartidos,qtdPartidos,percentualPartidos,totalInternet,qtdInternet,percentualInternet,totalRoni,qtdRoni,percentualRoni,totalProprios,qtdProprios,percentualProprios,totalReceitaOutCand,qtdReceitaOutCand,percentualReceitaOutCand,totalDoacaoFcc,qtdDoacaoFcc,percentualDoacaoFcc,sqPrestadorConta,sqEntregaPrestacao,graphQtdReceitaFin,graphVrReceitaFin,graphQtdReceitaFinFundo,graphVrReceitaFinFundo,graphPercentReceitaFinFundo,graphQtdReceitaFinOutros,graphVrReceitaFinOutros,graphPercentFinOutros,graphQtdReceitaFinFefc,graphVrReceitaFinFefc,graphPercentFinFefc,concentracaoDespesasR,rankingDoadoresR,rankingFornecedoresR,url_gov,id_tse,name,partido_y,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento,key_url
0,50000623171,CÉLIA SACRAMENTO,18,CÉLIA OLIVEIRA DE JESUS SACRAMENTO,FEM.,PRETA,Superior completo,Professor de Ensino Superior,14000000,7000000,SP,SÃO PAULO,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,REDE,**,Governador,1198000.00,Rede Sustentabilidade,SP-SÃO PAULO,REDE,Rede Sustentabilidade,REDE,18,NaN,NaN,14000000.0,7000000.0,14000000.0,50301.15,50301.15,0.0,30000.00,20000.00,301.15,0.0,0.0,50301.15,6,NaN,50301.15,6,100.00,0.00,0,0.00,0.00,0,0.00,0,0,0,50000.00,2,99.40,0,0,0,0,0,0.00,101.15,3,0.2,0.00,0,0.0,200,1,0.40,NaN,NaN,6,50301.15,1,30000.00,59.64,4,301.15,0.60,1,20000.00,39.76,50301.15,50301.15,38950.0,https://especiais.gazetadopovo.com.br/eleicoes...,50000623171,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP,celia-sacramento
1,50000622082,JOÃO HENRIQUE DURVAL CARNEIRO,28,JOÃO HENRIQUE DE BARRADAS CARNEIRO,MASC.,PARDA,Superior completo,Economista,14000000,7000000,BA,FEIRA DE SANTANA,BAHIA,https://divulgacandcontas.tse.jus.br/candidatu...,Não eleito,"BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS",PRTB / PSL,Governador,1502885.55,Partido Renovador Trabalhista Brasileiro,BA-FEIRA DE SANTANA,PRTB,Partido Renovador Trabalhista Brasileiro,PRTB,28,NaN,NaN,14000000.0,7000000.0,14000000.0,1000.00,1000.00,0.0,0.00,0.00,1000.00,0.0,0.0,1500.00,2,NaN,1000.00,1,66.67,500.00,1,33.33,1500.00,2,100.00,0,0,0,0.00,0,0.00,0,0,0,0,0,0.00,0.00,0,0.0,0.00,0,0.0,0,0,0.00,NaN,NaN,1,1000.00,0,0.00,0.00,1,1000.00,100.00,0,0.00,0.00,1500.00,1500.00,990.0,https://especiais.gazetadopovo.com.br/eleicoes...,50000622082,João Henrique Durval Carneiro,PRTB,0.58%,38960,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\t ...",João Henrique de Barradas Carneiro é candidato...,João Henrique Durval Carneiro,Masculino,62 anos,19/06/1959,Economista,Superior completo,Casado(a),Parda,Feira de Santana-BA,joao-henrique-durval-carneiro
2,50000608056,JOÃO SANTANA,15,JOÃO REIS SANTANA FILHO,MASC.,PARDA,Ensino M

In [73]:
df_gov.to_csv('clean_gov.csv', index=False, sep=';', encoding='iso-8859-1')

In [74]:
from fast_ml import eda

Anotaçoes quanto aos dados

In [75]:
summary_df = eda.df_info(df_gov)
display_all(summary_df)

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
id,int64,Numerical,7,"[50000623171, 50000622082, 50000608056, 500006...",0,0.0
nomeUrna,object,Categorical,7,"[CÉLIA SACRAMENTO, JOÃO HENRIQUE DURVAL CARNEI...",0,0.0
numero,int64,Numerical,7,"[18, 28, 15, 50, 29, 13, 25]",0,0.0
nomeCompleto,object,Categorical,7,"[CÉLIA OLIVEIRA DE JESUS SACRAMENTO, JOÃO HENR...",0,0.0
descricaoSexo,object,Categorical,2,"[FEM., MASC.]",0,0.0
descricaoCorRaca,object,Categorical,3,"[PRETA, PARDA, BRANCA]",0,0.0
grauInstrucao,object,Categorical,2,"[Superior completo, Ensino Médio completo]",0,0.0
ocupacao,object,Categorical,6,"[Professor de Ensino Superior, Economista, Apo...",0,0.0
gastoCampanha1T,int64,Numerical,1,[14000000],0,0.0
gastoCampanha2T,int64,Numerical,1,[7000000],0,0.0


In [80]:
eda.numerical_describe(df_gov)

,count,mean,std,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,95%,100%
id,7.00,50000612734.29,8584.76,50000600867.00,50000604973.40,50000607780.00,50000607987.00,50000608161.20,50000608319.00,50000614688.00,50000619563.60,50000621452.40,50000622517.60,50000622844.30,50000623171.00
numero,7.00,25.43,12.53,13.00,14.20,15.60,17.40,20.80,25.00,26.80,28.20,28.80,37.40,43.70,50.00
gastoCampanha1T,7.00,14000000.00,0.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00
gastoCampanha2T,7.00,7000000.00,0.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00
totalDeBens,7.00,874521.45,758612.05,0.00,189000.00,316108.28,319433.14,462051.82,674317.43,988526.97,1258977.11,1441908.44,1746093.62,1928499.67,2110905.72
nrCandidato,7.00,25.43,12.53,13.00,14.20,15.60,17.40,20.80,25.00,26.80,28.20,28.80,37.40,43.70,50.00
dividaCampanha,3.00,393825.27,682125.38,0.00,0.00,0.00,0.00,0.00,0.00,236295.16,472590.33,708885.49,945180.66,1063328.24,1181475.82
sobraFinanceira,3.00,2600.65,4163.95,26.54,95.38,164.22,233.07,301.91,370.75,1777.53,3184.31,4591.09,5997.87,6701.26,7404.65
limiteDeGasto1T,7.00,14000000.00,0.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00,14000000.00
limiteDeGasto2T,7.00,7000000.00,0.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00,7000000.00


In [81]:
eda.numerical_variable_detail(df_gov)

TypeError: numerical_variable_detail() missing 1 required positional argument: 'variable'

## Join City Data

In [42]:
print(len(df_gov_by_cities))
df_gov_by_cities.head(1)

2919


,url_source_gov,governor,url_the_city,name,percentage,votes
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Caetanos,"95,66%",6.089


In [41]:
print(len(df_all_city))
df_all_city.head(1)

417


,name,url_city,totalVotes,mascVotes,femnVotes
0,Salvador,https://especiais.gazetadopovo.com.br/eleicoes...,1827436,1022159,805276


In [40]:
# url_source_gov, url_the_city
df_city = pd.merge(df_gov_by_cities, df_all_city,
                   how="inner", left_on ="name", right_on="name")
print(len(df_city))

2919


In [46]:
list(df_city.iloc[0])

['https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/rui-costa/',
 'Rui Costa Eleito',
 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/caetanos-ba/governador/',
 'Caetanos',
 '95,66%',
 6.089,
 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/caetanos-ba/',
 9650,
 4591,
 5054]

In [ ]:
def last_part_url_as_key(txt):

In [48]:
atx = 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/rui-costa/'
atx.split('/')[-2]
    

'rui-costa'

In [49]:
df_city.head(1)

,url_source_gov,governor,url_the_city,name,percentage,votes,url_city,totalVotes,mascVotes,femnVotes
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Caetanos,"95,66%",6.089,https://especiais.gazetadopovo.com.br/eleicoes...,9650,4591,5054


In [50]:
df_city['key_url'] = df_city['url_source_gov'].apply(lambda x: x.split('/')[-2])
df_city.head(1)

,url_source_gov,governor,url_the_city,name,percentage,votes,url_city,totalVotes,mascVotes,femnVotes,key_url
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Caetanos,"95,66%",6.089,https://especiais.gazetadopovo.com.br/eleicoes...,9650,4591,5054,rui-costa


In [53]:
# df_city.drop('url_source_gov', inplace=True, axis=1)
df_city.drop('url_the_city', inplace=True, axis=1)
df_city.head(1)

,governor,name,percentage,votes,totalVotes,mascVotes,femnVotes,key_url
0,Rui Costa Eleito,Caetanos,"95,66%",6.089,9650,4591,5054,rui-costa


In [54]:
df_city['governor'] = df_city['governor'].apply(lambda x: x.replace(' Eleito','').strip())
df_city.head(1)

,governor,name,percentage,votes,totalVotes,mascVotes,femnVotes,key_url
0,Rui Costa,Caetanos,"95,66%",6.089,9650,4591,5054,rui-costa


In [55]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   governor    2919 non-null   object 
 1   name        2919 non-null   object 
 2   percentage  2919 non-null   object 
 3   votes       2919 non-null   float64
 4   totalVotes  2919 non-null   int64  
 5   mascVotes   2919 non-null   int64  
 6   femnVotes   2919 non-null   int64  
 7   key_url     2919 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 205.2+ KB


In [56]:
df_city.columns

Index(['governor', 'name', 'percentage', 'votes', 'totalVotes', 'mascVotes', 'femnVotes', 'key_url'], dtype='object')

In [57]:
df_city = df_city.reindex(columns=['key_url', 'governor', 'name', 'percentage', 'votes',
                         'totalVotes', 'mascVotes', 'femnVotes' ])
df_city.head(1)

,key_url,governor,name,percentage,votes,totalVotes,mascVotes,femnVotes
0,rui-costa,Rui Costa,Caetanos,"95,66%",6.089,9650,4591,5054


In [60]:
df_city['votes'] = df_city['votes'].apply(lambda x: str(x).replace('.','').strip())

In [68]:
df_city.head(1)

,key_url,governor,name,percentage,votes,totalVotes,mascVotes,femnVotes
0,rui-costa,Rui Costa,Caetanos,"95,66%",6089,9650,4591,5054


In [62]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key_url     2919 non-null   object
 1   governor    2919 non-null   object
 2   name        2919 non-null   object
 3   percentage  2919 non-null   object
 4   votes       2919 non-null   object
 5   totalVotes  2919 non-null   int64 
 6   mascVotes   2919 non-null   int64 
 7   femnVotes   2919 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 205.2+ KB


In [63]:
from fast_ml.utilities import reduce_memory_usage, display_all

In [64]:
df_city =  reduce_memory_usage(df_city, convert_to_category=False)

In [66]:
df_city['votes'] = df_city['votes'].astype(int)

In [67]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key_url     2919 non-null   object
 1   governor    2919 non-null   object
 2   name        2919 non-null   object
 3   percentage  2919 non-null   object
 4   votes       2919 non-null   int32 
 5   totalVotes  2919 non-null   int32 
 6   mascVotes   2919 non-null   int32 
 7   femnVotes   2919 non-null   int32 
dtypes: int32(4), object(4)
memory usage: 159.6+ KB


In [69]:
df_city.to_csv('clean_city.csv', index=False, sep=';', encoding='iso-8859-1')